In [7]:
import torch
from torch.autograd import Function
from torch.autograd import Variable as V
import torch as t
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython import display
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
device = torch.device("cuda")
device1 = torch.device("cpu")
import torch.optim as optim
import time
import pandas as pd
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn.modules import Module
from torch.nn.modules.lazy import LazyModuleMixin
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
#Mean Square Error
def MSE(pred,true):
    return np.mean((pred-true)**2)

#Root Mean Square Error
def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))

#Mean Absolute Error
def MAE(pred, true):
    return np.mean(np.abs(pred-true))

#Mean Absolute Percentage Error
def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

#Relative Squared Error
def RSE(pred, true):
    return np.sqrt(np.sum((true-pred)**2)) / np.sqrt(np.sum((true-true.mean())**2))

def CORR(pred, true):
    u = ((true-true.mean(0))*(pred-pred.mean(0))).sum(0) 
    d = np.sqrt(((true-true.mean(0))**2*(pred-pred.mean(0))**2).sum(0))
    return (u/d).mean(-1)

'''
def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))
'''

#Coefficient of Determination
def R2(pred, true):
    return r2_score(pred,true) 


def gamma(a):
    
    if a <= 0 and a %1 == 0:
        return float('inf')
    
    return math.gamma(a)



def matrix_fractional_differentiation_linear_plus(x,w,b,a):
    
    w_matrix_fractional_differentiation = x
    
    w_rows, w_cols = w.size()
    x_rows, x_cols = x.size()
    x = ((t.sum(x,dim=0)/x_rows).view(1,-1))
   
    y = F.linear(x,w.T,b)  
    f1 = t.mul(x,w.T).T
    c_partial = y - f1
    
    #x
    xx = t.abs(x)
    x_partial = t.mul(c_partial.T,xx**(-a)/gamma(1-a)).T
    x_main = t.mul(w.T,xx**(1-a)/gamma(2-a)).T
    
    '''
    #w
    ww = t.abs(w)
    w_partial = t.mul(c_partial,ww**(-a)/gamma(1-a))
    w_main = t.mul(x,ww.T**(1-a)/gamma(2-a)).T 
    w_matrix_fractional_differentiation = (t.sum(w_main+w_partial,dim=1)/w_cols).view(1,-1)
    '''
    #b
    bb = t.abs(b)
    b_c_partial = y - b
    b_partial = t.mul(b_c_partial,bb**(-a)/gamma(1-a))
    b_main = bb**(1-a)/gamma(2-a)
    
    return x_main+x_partial, w_matrix_fractional_differentiation, b_main+b_partial

#This is not an important Class; its purpose is solely to obtain the weight matrix W and bias b.
#We have rearranged the positions of 'in_features' and 'out_features' for ease of future use.
class Linear(Module):

    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((in_features, out_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight.T, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
    
    
def split(X,y):
    X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333,shuffle=False)
    return X_train,X_val,X_test,y_train,y_val,y_test


#ifogd_MSELoss
class MSELoss_fractional(Function):
    
    def forward(ctx,input1,target):
        loss = nn.MSELoss()
        ctx.save_for_backward(input1,target)
        return loss(input1,target)
    
    def backward(ctx,grad_outputs):
        input1,target = ctx.saved_tensors
        t0, t1 = t.tensor([0.]),t.tensor([1.])
        t0 = t0.to(device)
        t1 = t1.to(device)
        y1 = t.where(input1 - target <= 0, t0, t1)
        y_pred_f = t.abs(input1 - target)
        grad_output1 = gamma(3)*y_pred_f**(2-a)/gamma(3-a)*y1
        y2 = t.where(input1 - target > 0, t0, t1)
        grad_output2 = 2*(input1 - target)*y2

        grad_output = (grad_output1 + grad_output2)/input1.numel()
        
        return grad_output,None
    
#ifogd_Linear
class Linear_fractional(Function):
    
    def forward(ctx,input1,w,b):
        ctx.save_for_backward(input1,w,b)
        outputs = input1@w + b
        return outputs
    
    def backward(ctx,grad_outputs):
        input1,w,b = ctx.saved_tensors
        #w4:x_matrix_fractional_differentiation
        #x4:w_matrix_fractional_differentiation
        #b4:b_matrix_fractional_differentiation
        w4, x4, b4 = matrix_fractional_differentiation_linear_plus(input1,w,b,a)   #IFOGD
        
        x_grad = torch.mm(grad_outputs,w4.T)
        w_grad = torch.mm(x4.T,grad_outputs)
        b_grad = torch.mul(grad_outputs,b4)
        return x_grad,w_grad,b_grad
    

In [9]:
#GSPC，DJI，IXIC，FTSE，GDAXI，N225 data preprocessing
time1 = time.time()
slide_windows_size = 36  #i.e.,input length
stock_array = ['GSPC','DJI','IXIC','FTSE','GDAXI','N225']   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
#stock_array = ['ETTh1','ETTh2','ETTm1','ETTm2']   #ETTh1，ETTh2，ETTm1，ETTm2
for stock in stock_array:
    #stock = 'DJI'   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
    pre_days = 48     #i.e.,prediction lengths,48,96,192,384
    while pre_days <= 384:  
        df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
        del df_DJIA['Date']
        #del df_DJIA['date']        #ETT
        sca_DJIA = scaler.fit_transform(df_DJIA)
        label = t.tensor([row[4] for row in sca_DJIA][slide_windows_size:]).view(-1,1)
        #label = t.tensor([row[6] for row in sca_DJIA][slide_windows_size:]).view(-1,1)  #ETT
        sca_DJIA = t.tensor(sca_DJIA[:-pre_days])
        #label
        deq_DJIA = deque(maxlen=pre_days)
        DJIA_label = []
        for i in label:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==pre_days:
                DJIA_label.append(list(deq_DJIA))
        DJIA_label = t.Tensor(DJIA_label).view(-1,pre_days)
        #X
        deq_DJIA = deque(maxlen=slide_windows_size)
        DJIA_X = []
        for i in sca_DJIA:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==slide_windows_size:
                DJIA_X.append(list(deq_DJIA))
        DJIA_X = t.Tensor(DJIA_X)
        X_DJIA, y_DJIA = DJIA_X.to(device), DJIA_label.to(device)
        X_train,X_val,X_test,y_train,y_val,y_test = split(X_DJIA,y_DJIA)   #7:2:1

        class Net(nn.Module):
            def __init__(self,):
                super(Net,self).__init__()
                self.linear1 = Linear(slide_windows_size*5,128)
                #self.linear1 = Linear(slide_windows_size*7,128)  #ETT
                self.linear2 = Linear(128,64)
                self.linear3 = Linear(64,32)
                self.linear4 = Linear(32,pre_days)
                self.w1,self.b1 = self.linear1.weight,self.linear1.bias
                self.w2,self.b2 = self.linear2.weight,self.linear2.bias
                self.w3,self.b3 = self.linear3.weight,self.linear3.bias
                self.w4,self.b4 = self.linear4.weight,self.linear4.bias

            def forward(self,x):
                x = x.view(-1,slide_windows_size*5)
                #x = x.view(-1,slide_windows_size*7)   #ETT
                y2 = Linear_fractional.apply(x,self.w1,self.b1)   
                y3 = Linear_fractional.apply(y2,self.w2,self.b2)   
                y4 = Linear_fractional.apply(y3,self.w3,self.b3) 
                y5 = Linear_fractional.apply(y4,self.w4,self.b4)  
                return y5

        net = Net()
        torch.save(net.state_dict(), r'./model/Adadelta_Stock/'+stock+'_'+str(pre_days)+'_net.pth')    

        a = 0.6
        lr =1e-2   #adadelta:1e-2
        epoch = 1000
        batch_size = 256
        train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11 = [],[],[],[],[]  #collecting train set loss
        while a < 1.1:
            a = round(a + 0.1,1)
            best_loss = 10
            net = Net()
            net.load_state_dict(torch.load(r'./model/Adadelta_Stock/'+stock+'_'+str(pre_days)+'_net.pth'))
            optimizer = torch.optim.Adadelta(net.parameters(),lr=lr)
            net.to(device)
            for i in range(epoch):
                loss_sum = 0
                net.train()  
                start = np.random.randint(11,size=1)[0]  #Generate a random integer between 0 and 10
                while start+batch_size<len(X_train):
                    X_batch = X_train[start:start+batch_size]
                    y_batch = y_train[start:start+batch_size]
                    start = start+batch_size
                    out = net(X_batch)
                    loss = MSELoss_fractional.apply(out,y_batch) 
                    loss.backward()
                    loss_sum += loss
                    optimizer.step()
                    net.zero_grad()
                if a==0.7:
                    train_loss_07.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.8:
                    train_loss_08.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.9:
                    train_loss_09.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.0:
                    train_loss_10.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.1:
                    train_loss_11.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                net.eval()
                with torch.no_grad():
                    out = net(X_val)
                    MSE_val = MSE(y_val.cpu().detach().numpy(),out.cpu().detach().numpy())   #MSE，MAPE
                    if best_loss > MSE_val:
                        best_loss = MSE_val
                        torch.save(net.state_dict(), r'./model/Adadelta_Stock/'+stock+'_net_fractional_'+str(a).replace(".", "_")+'_'+str(pre_days)+'.pth')
                        print('epoch:{0}'.format(i+1)+'\n'+'Validation Loss:{0}'.format(MSE_val))
            print('the order'+str(a)+'finished!')
        if pre_days==48:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adadelta_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==96:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adadelta_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==192:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adadelta_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==384:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adadelta_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        pre_days = pre_days*2
time2 = time.time()
print(time2 - time1)

epoch:1
Validation Loss:0.9805399179458618
epoch:2
Validation Loss:0.9758314490318298
epoch:3
Validation Loss:0.9724355340003967
epoch:4
Validation Loss:0.9698309898376465
epoch:5
Validation Loss:0.969201922416687
epoch:6
Validation Loss:0.967949390411377
epoch:7
Validation Loss:0.9651494026184082
epoch:8
Validation Loss:0.9599923491477966
epoch:9
Validation Loss:0.9576743841171265
epoch:10
Validation Loss:0.9539714455604553
epoch:11
Validation Loss:0.9499133229255676
epoch:12
Validation Loss:0.9450879096984863
epoch:13
Validation Loss:0.9394290447235107
epoch:14
Validation Loss:0.9312841296195984
epoch:15
Validation Loss:0.9267290234565735
epoch:16
Validation Loss:0.9224120378494263
epoch:17
Validation Loss:0.9140215516090393
epoch:18
Validation Loss:0.9055120944976807
epoch:19
Validation Loss:0.896395742893219
epoch:20
Validation Loss:0.8871711492538452
epoch:21
Validation Loss:0.8774813413619995
epoch:22
Validation Loss:0.8693299889564514
epoch:23
Validation Loss:0.8603249192237854


epoch:817
Validation Loss:0.061531804502010345
epoch:818
Validation Loss:0.061490464955568314
epoch:819
Validation Loss:0.061415184289216995
epoch:820
Validation Loss:0.06137542054057121
epoch:821
Validation Loss:0.061316825449466705
epoch:822
Validation Loss:0.06120957434177399
epoch:823
Validation Loss:0.06115341559052467
epoch:824
Validation Loss:0.0611066073179245
epoch:825
Validation Loss:0.06101476773619652
epoch:826
Validation Loss:0.06095600873231888
epoch:827
Validation Loss:0.0609099455177784
epoch:830
Validation Loss:0.06086268275976181
epoch:831
Validation Loss:0.06084791570901871
epoch:832
Validation Loss:0.06080591678619385
epoch:833
Validation Loss:0.060734041035175323
epoch:834
Validation Loss:0.06067859008908272
epoch:835
Validation Loss:0.060602232813835144
epoch:836
Validation Loss:0.06051751598715782
epoch:837
Validation Loss:0.06045831739902496
epoch:838
Validation Loss:0.060403190553188324
epoch:839
Validation Loss:0.06034907326102257
epoch:840
Validation Loss:0.0

epoch:84
Validation Loss:0.5105814933776855
epoch:85
Validation Loss:0.4936598539352417
epoch:86
Validation Loss:0.476106196641922
epoch:87
Validation Loss:0.45831382274627686
epoch:88
Validation Loss:0.44067010283470154
epoch:89
Validation Loss:0.42140910029411316
epoch:90
Validation Loss:0.4018619656562805
epoch:91
Validation Loss:0.38247302174568176
epoch:92
Validation Loss:0.363717645406723
epoch:93
Validation Loss:0.34643784165382385
epoch:94
Validation Loss:0.3299688398838043
epoch:95
Validation Loss:0.3140907287597656
epoch:96
Validation Loss:0.29859983921051025
epoch:97
Validation Loss:0.2836611568927765
epoch:98
Validation Loss:0.26926490664482117
epoch:99
Validation Loss:0.25562015175819397
1.2173616
epoch:100
Validation Loss:0.24248653650283813
epoch:101
Validation Loss:0.23003408312797546
epoch:102
Validation Loss:0.2184237837791443
epoch:103
Validation Loss:0.20746725797653198
epoch:104
Validation Loss:0.19742678105831146
epoch:105
Validation Loss:0.18807198107242584
epoch

epoch:369
Validation Loss:0.06860222667455673
epoch:371
Validation Loss:0.06818249076604843
epoch:384
Validation Loss:0.06807121634483337
epoch:385
Validation Loss:0.0677700862288475
epoch:387
Validation Loss:0.0676707848906517
epoch:388
Validation Loss:0.06750363111495972
epoch:389
Validation Loss:0.0668000653386116
0.14773229
epoch:401
Validation Loss:0.06671271473169327
epoch:402
Validation Loss:0.06641963124275208
epoch:404
Validation Loss:0.0663890615105629
epoch:406
Validation Loss:0.06611756235361099
epoch:407
Validation Loss:0.06581657379865646
epoch:409
Validation Loss:0.06525684148073196
epoch:424
Validation Loss:0.06496743857860565
epoch:432
Validation Loss:0.06471861153841019
epoch:437
Validation Loss:0.06470627337694168
epoch:438
Validation Loss:0.06423487514257431
epoch:440
Validation Loss:0.063967764377594
epoch:441
Validation Loss:0.06318990141153336
epoch:444
Validation Loss:0.06277022510766983
0.14292896
epoch:459
Validation Loss:0.0625600814819336
epoch:460
Validatio

epoch:41
Validation Loss:0.6599856019020081
epoch:42
Validation Loss:0.6549813747406006
epoch:43
Validation Loss:0.6524728536605835
epoch:44
Validation Loss:0.6484523415565491
epoch:45
Validation Loss:0.6441591382026672
epoch:46
Validation Loss:0.6383137702941895
epoch:47
Validation Loss:0.6324344277381897
epoch:48
Validation Loss:0.627402126789093
epoch:49
Validation Loss:0.6229123473167419
4.1604366
epoch:50
Validation Loss:0.6194862723350525
epoch:51
Validation Loss:0.6157917976379395
epoch:52
Validation Loss:0.6120747923851013
epoch:53
Validation Loss:0.6077305674552917
epoch:54
Validation Loss:0.6038531064987183
epoch:55
Validation Loss:0.6022496223449707
epoch:56
Validation Loss:0.6013101935386658
epoch:58
Validation Loss:0.597690761089325
epoch:59
Validation Loss:0.593466579914093
epoch:60
Validation Loss:0.5892056226730347
epoch:61
Validation Loss:0.584402322769165
epoch:62
Validation Loss:0.579779326915741
epoch:63
Validation Loss:0.5740683674812317
epoch:64
Validation Loss:0.

...

epoch:10
Validation Loss:0.823043704032898
epoch:11
Validation Loss:0.8223862051963806
epoch:12
Validation Loss:0.821744441986084
epoch:13
Validation Loss:0.8211097121238708
epoch:14
Validation Loss:0.8205718398094177
epoch:15
Validation Loss:0.8196569681167603
epoch:16
Validation Loss:0.8188276886940002
epoch:17
Validation Loss:0.81783127784729
epoch:18
Validation Loss:0.817407488822937
epoch:19
Validation Loss:0.8168647885322571
epoch:20
Validation Loss:0.8163231015205383
epoch:21
Validation Loss:0.8157579302787781
epoch:22
Validation Loss:0.8152246475219727
epoch:23
Validation Loss:0.8144884705543518
epoch:24
Validation Loss:0.813758373260498
epoch:25
Validation Loss:0.8130270838737488
epoch:26
Validation Loss:0.8122977614402771
epoch:27
Validation Loss:0.8114163875579834
epoch:28
Validation Loss:0.8104275465011597
epoch:29
Validation Loss:0.8094371557235718
epoch:30
Validation Loss:0.8086431622505188
epoch:31
Validation Loss:0.8080933094024658
epoch:32
Validation Loss:0.80724090337

epoch:195
Validation Loss:0.2909371852874756
epoch:196
Validation Loss:0.2908724546432495
epoch:197
Validation Loss:0.290824830532074
epoch:198
Validation Loss:0.2907768487930298
epoch:199
Validation Loss:0.29069262742996216
1.9240185
epoch:200
Validation Loss:0.29038286209106445
epoch:223
Validation Loss:0.2903725206851959
epoch:225
Validation Loss:0.29037001729011536
epoch:235
Validation Loss:0.2903265655040741
epoch:236
Validation Loss:0.2903074026107788
epoch:243
Validation Loss:0.2903052866458893
epoch:247
Validation Loss:0.2902829945087433
epoch:248
Validation Loss:0.2902615964412689
epoch:249
Validation Loss:0.29025280475616455
1.9263299
epoch:254
Validation Loss:0.2902469038963318
epoch:256
Validation Loss:0.29023799300193787
epoch:257
Validation Loss:0.2901850938796997
epoch:258
Validation Loss:0.2901678681373596
epoch:259
Validation Loss:0.29012152552604675
epoch:260
Validation Loss:0.29009145498275757
epoch:261
Validation Loss:0.29003748297691345
epoch:263
Validation Loss:0.

epoch:110
Validation Loss:0.34434622526168823
epoch:111
Validation Loss:0.3415147364139557
epoch:112
Validation Loss:0.33895984292030334
epoch:113
Validation Loss:0.3365785777568817
epoch:114
Validation Loss:0.33405181765556335
epoch:115
Validation Loss:0.3318198025226593
epoch:116
Validation Loss:0.32989194989204407
epoch:117
Validation Loss:0.3278626799583435
epoch:118
Validation Loss:0.3260350525379181
epoch:119
Validation Loss:0.3244336247444153
epoch:120
Validation Loss:0.32273203134536743
epoch:121
Validation Loss:0.32124313712120056
epoch:122
Validation Loss:0.3193720877170563
epoch:123
Validation Loss:0.31818270683288574
epoch:124
Validation Loss:0.31666815280914307
epoch:125
Validation Loss:0.3152468800544739
epoch:126
Validation Loss:0.3143489360809326
epoch:127
Validation Loss:0.31339144706726074
epoch:128
Validation Loss:0.31252342462539673
epoch:129
Validation Loss:0.3115706741809845
epoch:130
Validation Loss:0.3108071982860565
epoch:131
Validation Loss:0.3099522292613983


epoch:30
Validation Loss:0.6696992516517639
epoch:31
Validation Loss:0.6564534902572632
epoch:32
Validation Loss:0.642756998538971
epoch:33
Validation Loss:0.6286093592643738
epoch:34
Validation Loss:0.614095151424408
epoch:35
Validation Loss:0.5993664264678955
epoch:36
Validation Loss:0.5843989253044128
epoch:37
Validation Loss:0.5693359971046448
epoch:38
Validation Loss:0.5543121099472046
epoch:39
Validation Loss:0.5394234657287598
epoch:40
Validation Loss:0.5247941613197327
epoch:41
Validation Loss:0.5104653239250183
epoch:42
Validation Loss:0.4964601695537567
epoch:43
Validation Loss:0.4829309582710266
epoch:44
Validation Loss:0.46987444162368774
epoch:45
Validation Loss:0.45728492736816406
epoch:46
Validation Loss:0.4453429579734802
epoch:47
Validation Loss:0.43400678038597107
epoch:48
Validation Loss:0.42326653003692627
epoch:49
Validation Loss:0.41316458582878113
2.781672
epoch:50
Validation Loss:0.40374165773391724
epoch:51
Validation Loss:0.39483797550201416
epoch:52
Validatio

epoch:95
Validation Loss:0.7174673080444336
epoch:96
Validation Loss:0.7150651216506958
epoch:97
Validation Loss:0.7125967144966125
epoch:98
Validation Loss:0.710042417049408
epoch:99
Validation Loss:0.7072122693061829
4.717874
epoch:100
Validation Loss:0.70317143201828
epoch:101
Validation Loss:0.6994092464447021
epoch:102
Validation Loss:0.6951571106910706
epoch:103
Validation Loss:0.6916499137878418
epoch:104
Validation Loss:0.68890780210495
epoch:105
Validation Loss:0.6846693754196167
epoch:106
Validation Loss:0.6799237728118896
epoch:107
Validation Loss:0.6748669147491455
epoch:108
Validation Loss:0.6706138253211975
epoch:109
Validation Loss:0.6662507653236389
epoch:110
Validation Loss:0.6617119908332825
epoch:111
Validation Loss:0.6567069292068481
epoch:112
Validation Loss:0.6511788964271545
epoch:113
Validation Loss:0.6455009579658508
epoch:114
Validation Loss:0.6401816010475159
epoch:115
Validation Loss:0.6349887251853943
epoch:116
Validation Loss:0.6296157836914062
epoch:117
V